# Заполнение пропусков в данных
1. Заполнить отсутствующие значения по погоде интерполяционными данными.
2. Посчитать модель линейной регрессии по первому зданию и найти ее точность.

In [4]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [5]:
buildings = pd.read_csv("../data/buildings.csv")
weather = pd.read_csv("../data/weather.csv")
energy = pd.read_csv("../data/train.0.csv")

## Данные по 0-му зданию

In [6]:
energy = energy[energy["building_id"]==0]
print(energy["building_id"].unique())
print(energy.info())

[0]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8784 entries, 0 to 12059506
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   building_id    8784 non-null   int64  
 1   meter          8784 non-null   int64  
 2   timestamp      8784 non-null   object 
 3   meter_reading  8784 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 343.1+ KB
None


## Объединение данных

In [7]:
energy = pd.merge(
    left=energy,
    right=buildings,
    how="left",
    left_on="building_id",
    right_on="building_id",
)
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(
    left=energy,
    right=weather,
    how="left",
    left_index=True,
    right_index=True,
)
energy.reset_index(inplace=True)
energy = energy.drop(
    columns=[
        "meter",
        "site_id",
    ],
    axis=1
)

del buildings
del weather
print(energy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   timestamp           8784 non-null   object 
 1   building_id         8784 non-null   int64  
 2   meter_reading       8784 non-null   float64
 3   primary_use         8784 non-null   object 
 4   square_feet         8784 non-null   int64  
 5   year_built          8784 non-null   float64
 6   floor_count         0 non-null      float64
 7   air_temperature     8781 non-null   float64
 8   cloud_coverage      4954 non-null   float64
 9   dew_temperature     8781 non-null   float64
 10  precip_depth_1_hr   8783 non-null   float64
 11  sea_level_pressure  8699 non-null   float64
 12  wind_direction      8534 non-null   float64
 13  wind_speed          8784 non-null   float64
dtypes: float64(10), int64(2), object(2)
memory usage: 960.9+ KB
None


## Оптимизация памяти

In [8]:
def reduce_mem_usage(df: pd.DataFrame) -> pd.DataFrame:
    start_mem = df.memory_usage().sum()/1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
            
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Потребление памяти меньше на - {round(start_mem - end_mem, 2)} Мб (минус {round(100*(start_mem - end_mem)/start_mem, 1)}%)")
    return df

In [9]:
energy = reduce_mem_usage(energy)

Потребление памяти меньше на - 0.62 Мб (минус 66.1%)
